<a href="https://colab.research.google.com/github/deRockerTom/dataFrenchWords/blob/main/compressFrenchFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/deRockerTom/dataFrenchWords

Cloning into 'dataFrenchWords'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 19 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [88]:
def convertToBinary(number):
  binary = ''
  while number > 0:
    binary = str(number % 2) + binary
    number = number // 2
  return binary
def convertToNineBits(bits):
  n = len(bits)
  if n < 9:
    k = 9 - n
    return str('0') * k + bits
  elif n == 9:
    return bits
  else:
    print("Attention : un des encodages est trop grand (>9)")

convertToNineBits(convertToBinary(255))

'011111111'

In [89]:
import csv

# open csv
dic = {}
with open('dataFrenchWords/french-word-list-verbs.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-adjectives.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-nouns.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

# Ponctuation :

dic[', '] = 150000
dic['. '] = 150000

dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True))
dic = {k: dic[k] for k in list(dic)[:256]} # In order to code the dictionnary on only 2 other bytes with most frequent words
offset = 256
i = 0
for key in dic:
  dic[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1

print(dic)

reverted_dic = {v: k for k, v in dic.items()}

print(reverted_dic)

{'est': '100000000', 'ai': '100000001', 'autre': '100000010', 'fai': '100000011', 'peu': '100000100', 'nouveau': '100000101', 'grand': '100000110', 'bon': '100000111', 'petit': '100001000', ', ': '100001001', '. ': '100001010', 'm?me': '100001011', 'va': '100001100', 'doi': '100001101', 'di': '100001110', 'dernier': '100001111', 'voi': '100010000', 'seul': '100010001', 'pren': '100010010', 'beau': '100010011', 'sai': '100010100', 'met': '100010101', 'veu': '100010110', 'jeune': '100010111', 'meilleur': '100011000', 'français': '100011001', 'différent': '100011010', 'trouv': '100011011', 'nombreux': '100011100', 'important': '100011101', 'juste': '100011110', 'présent': '100011111', 'donn': '100100000', 'ven': '100100001', 'pass': '100100010', 'fall': '100100011', 'permett': '100100100', 'vrai': '100100101', 'part': '100100110', 'politique': '100100111', 'propre': '100101000', 'social': '100101001', 'possible': '100101010', 'général': '100101011', 'national': '100101100', 'utilis': '100

In [90]:
# class to build a tree defining all the possible  string in dic
class SuffixTree():
  class Node():
    def __init__(self,char):
      self.char = char
      self.out = {}
      self.isFinal = False
    def __str__(self, level=0):
      ret = "  " * level + str(self.char) + "\n"
      for c in self.out:
        ret += self.out[c].__str__(level + 1)
      return ret

  def __init__(self, dic):
    self.root = self.Node(None)
    self.root.isFinal = True
    for key in dic:
      cursor = self.root
      for c in key:
        if c in cursor.out:
          cursor = cursor.out[c]
        else:
          node = self.Node(c)
          cursor.out[c] = node
          cursor = node
      cursor.isFinal = True
  def longestPrefix(self, s):
    cursor = self.root
    word = ''
    temp = ''
    end = False
    i = 0
    length = len(s)
    while not(end) and i < length:
      if s[i] in cursor.out:
        temp = temp + s[i]
        cursor = cursor.out[s[i]]
        if cursor.isFinal:
          word = temp
        i += 1
      else:
        end = True
    return word
  def __str__(self):
    return f'{self.root}'

s = SuffixTree({'ab' : 1, 'abc': 2, 'coucou' : 5, 'aba' : 4, 'aaa' : 8})
print(s)
s.longestPrefix('abdc')
    


None
  a
    b
      c
      a
    a
      a
  c
    o
      u
        c
          o
            u



'ab'

In [91]:
tree = SuffixTree(dic)

In [97]:
# First implementation

string = 'ceci est un test d\'implémentation'

def compress(string, tree, dic):
  i = 0
  ret = ''
  while i<len(string):
    temp = tree.longestPrefix(string[i:])
    n = len(temp)
    if n == 0:
      bit = convertToNineBits(convertToBinary(ord(string[i])))
      ret += bit
      i += 1
    else:
      bit = dic[temp]
      ret += bit
      i += n
  return ret

compressed = compress(string, tree, dic)
print(compressed)

def decompress(string, dic):
  ret = ''
  for i in range(0, len(string) - 1, 9):
    if int(string[i : i + 9], 2) > 255:
      ret += dic[string[i : i + 9]]
    else:
      ret += chr(int(string[i : i + 9], 2))
  return ret

decompressed = decompress(compressed, reverted_dic)
print(decompressed)

print(f'Le fichier avant décompression faisait {len(string) * 8} octets et celui compressé fait {len(compressed)} octets')
print(f'Le taux de compression est donc de {len(string) * 8 / len(compressed)}')
  


001100011001100101001100011001101001000100000100000000000100000001110101001101110000100000001110100100000000000100000001100100000100111001101001001101101001110000001101100011101001001101101001100101001101110001110100001100001001110100001101001001101111001101110
ceci est un test d'implémentation
Le fichier avant décompression faisait 264 octets et celui compressé fait 261 octets
Le taux de compression est donc de 1.0114942528735633
